## Libraries

In [26]:
import pandas as pd
import numpy as np
import scipy.stats as stat

from math import sqrt
from mlgear.utils import show, display_columns
from surveyweights import normalize_weights


def margin_of_error(n=None, sd=None, p=None, type='proportion', interval_size=0.95):
    z_lookup = {0.8: 1.28, 0.85: 1.44, 0.9: 1.65, 0.95: 1.96, 0.99: 2.58}
    if interval_size not in z_lookup.keys():
        raise ValueError('{} not a valid `interval_size` - must be {}'.format(interval_size,
                                                                              ', '.join(list(z_lookup.keys()))))
    if type == 'proportion':
        se = sqrt(p * (1 - p)) / sqrt(n)
    elif type == 'continuous':
        se = sd / sqrt(n)
    else:
        raise ValueError('{} not a valid `type` - must be proportion or continuous')
    
    z = z_lookup[interval_size]
    return se * z


def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<0.1%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def calc_result(for_vote, against_vote, n, interval=0.8):
    GENERAL_POLLING_ERROR = 8.0
    TIME_SHIFT_ERROR = 4.0
    N_SIMS = 10000000
    
    for_moe = margin_of_error(n=n, p=for_vote/100, interval_size=interval)
    against_moe = margin_of_error(n=n, p=against_vote/100, interval_size=interval)
    undecided = 100 - for_vote - against_vote
    mean = for_vote + undecided * 0.25
    raw_moe = for_moe * 100 + against_moe * 100
    
    allocate_undecided = undecided * 0.4
    margin = raw_moe + allocate_undecided + GENERAL_POLLING_ERROR + TIME_SHIFT_ERROR
    
    cdf_value = 0.5 + 0.5 * interval
    normed_sigma = stat.norm.ppf(cdf_value)
    sigma = margin / 100 / normed_sigma
    
    sims = np.random.normal(mean / 100, sigma, N_SIMS)
    chance_pass = np.sum([s > 0.5 for s in sims]) / N_SIMS
    low, high = np.percentile(sims, [20, 80]) * 100
    
    return {'mean': mean, 'high': high, 'low': low, 'n': n,
            'raw_moe': raw_moe, 'margin': margin, 'sigma': sigma, 'chance_pass': chance_pass}


def print_result(mean, high, low, n, raw_moe, margin, sigma, chance_pass):
    mean = np.round(mean, 1)
    first = np.round(high, 1)
    second = np.round(low, 1)
    sigma = np.round(sigma * 100, 1)
    raw_moe = np.round(raw_moe, 1)
    margin = np.round(margin, 1)
    chance_pass = print_pct(chance_pass, 1)
    if second < first:
        _ = first
        first = second
        second = _
    if second > 100:
        second = 100
    if first < 0:
        first = 0
    print(('Result {} (80% CI: {} to {}) (N={}) (raw_moe={}pts, margin={}pts, '
           'sigma={}pts) ({} likely to pass)').format(mean,
                                                      first,
                                                      second,
                                                      n,
                                                      raw_moe,
                                                      margin,
                                                      sigma,
                                                      chance_pass))
    print(('{} (80% CI: {} to {}) ({})').format(mean,
                                                first,
                                                second,
                                                chance_pass))
    print('-')
    

def calc_result_tb(biden_vote, trump_vote, n, interval=0.8):
    GENERAL_POLLING_ERROR = 3.0
    TIME_SHIFT_ERROR = 1.0
    N_SIMS = 100000
    
    biden_moe = margin_of_error(n=n, p=biden_vote/100, interval_size=interval)
    trump_moe = margin_of_error(n=n, p=trump_vote/100, interval_size=interval)
    undecided = (100 - biden_vote - trump_vote) / 2

    biden_mean = biden_vote + undecided * 0.25
    biden_raw_moe = biden_moe * 100
    biden_allocate_undecided = undecided * 0.4
    biden_margin = biden_raw_moe + biden_allocate_undecided + GENERAL_POLLING_ERROR + TIME_SHIFT_ERROR
    
    trump_mean = trump_vote + undecided * 0.25
    trump_raw_moe = trump_moe * 100
    trump_allocate_undecided = undecided * 0.4
    trump_margin = trump_raw_moe + trump_allocate_undecided + GENERAL_POLLING_ERROR + TIME_SHIFT_ERROR
    
    cdf_value = 0.5 + 0.5 * interval
    normed_sigma = stat.norm.ppf(cdf_value)
    
    biden_sigma = biden_margin / 100 / normed_sigma
    biden_sims = np.random.normal(biden_mean / 100, biden_sigma, N_SIMS)
    
    trump_sigma = trump_margin / 100 / normed_sigma
    trump_sims = np.random.normal(trump_mean / 100, trump_sigma, N_SIMS)
    
    chance_pass = np.sum([sim[0] > sim[1] for sim in zip(biden_sims, trump_sims)]) / N_SIMS
    
    low, high = np.percentile(biden_sims - trump_sims, [20, 80]) * 100
    
    return {'mean': biden_mean - trump_mean, 'high': high, 'low': low, 'n': n,
            'raw_moe': biden_raw_moe + trump_raw_moe,
            'margin': biden_margin + trump_margin,
            'sigma': np.mean([biden_sigma, trump_sigma]),
            'chance_pass': chance_pass}


def print_result_tb(mean, high, low, n, raw_moe, margin, sigma, chance_pass):
    mean = np.round(mean, 1)
    first = np.round(high, 1)
    second = np.round(low, 1)
    sigma = np.round(sigma * 100, 1)
    raw_moe = np.round(raw_moe, 1)
    margin = np.round(margin, 1)
    chance_pass = print_pct(chance_pass, 1)
    if second < first:
        _ = first
        first = second
        second = _
    if second > 100:
        second = 100
    if first < -100:
        first = -100
    print(('Result Biden {} (80% CI: {} to {}) (Weighted N={}) (raw_moe={}pts, margin={}pts, '
           'sigma={}pts) (Biden {} likely to win)').format(mean,
                                                           first,
                                                           second,
                                                           n,
                                                           raw_moe,
                                                           margin,
                                                           sigma,
                                                           chance_pass))
    print('-')

## Load Processed Data

In [27]:
ca_national_survey = pd.read_csv('repsonses_processed_ca_weighted.csv').fillna('Not presented')
ca_state_survey = pd.read_csv('repsonses_processed_ca_state_ca_weighted.csv').fillna('Not presented')

## California Trump-Clinton

In [28]:
options = ['Donald Trump', 'Hillary Clinton', 'Other']
survey_ = ca_national_survey.loc[ca_national_survey['vote2016'].isin(options)].copy()
survey_['weight'] = normalize_weights(survey_['weight'])
survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
survey_['vote2016'].value_counts(normalize=True) * survey_.groupby('vote2016')['weight'].mean() * 100

Donald Trump       31.672972
Hillary Clinton    61.684774
Other               6.642253
dtype: float64

In [29]:
options = ['Donald Trump', 'Hillary Clinton', 'Other']
survey_ = ca_state_survey.loc[ca_state_survey['vote2016'].isin(options)].copy()
survey_['weight'] = normalize_weights(survey_['weight'])
survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
survey_['vote2016'].value_counts(normalize=True) * survey_.groupby('vote2016')['weight'].mean() * 100

Donald Trump       30.913487
Hillary Clinton    62.621095
Other               6.465418
dtype: float64

## California Trump-Biden

In [30]:
options = ['Joe Biden, the Democrat', 'Donald Trump, the Republican', 'Another candidate', 'Not decided']
survey_ = ca_national_survey.loc[ca_national_survey['vote_trump_biden'].isin(options)].copy()
survey_['weight'] = normalize_weights(survey_['weight'])
survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
    
print('## NATIONAL CA-WEIGHTED ##')
weighted_n = int(np.round(survey_['weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_trump_biden'].value_counts(normalize=True) * survey_.groupby('vote_trump_biden')['weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result_tb(**calc_result_tb(biden_vote=votes['Joe Biden, the Democrat'],
                                 trump_vote=votes['Donald Trump, the Republican'],
                                 n=weighted_n))

print('## NATIONAL CA-WEIGHTED + RV ##')
rv_weighted_n = int(np.round(survey_['rv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_trump_biden'].value_counts(normalize=True) * survey_.groupby('vote_trump_biden')['rv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result_tb(**calc_result_tb(biden_vote=votes['Joe Biden, the Democrat'],
                                 trump_vote=votes['Donald Trump, the Republican'],
                                 n=rv_weighted_n))

print('## NATIONAL CA-WEIGHTED + LV ##')
lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_trump_biden'].value_counts(normalize=True) * survey_.groupby('vote_trump_biden')['lv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result_tb(**calc_result_tb(biden_vote=votes['Joe Biden, the Democrat'],
                                 trump_vote=votes['Donald Trump, the Republican'],
                                 n=lv_weighted_n))


print('## CALIFORNIA UNWEIGHTED ##')
survey_ = ca_state_survey.loc[ca_state_survey['vote_trump_biden'].isin(options)].copy()
survey_['weight'] = normalize_weights(survey_['weight'])
survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
n = len(survey_)
votes = survey_['vote_trump_biden'].value_counts(normalize=True) * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result_tb(**calc_result_tb(biden_vote=votes['Joe Biden, the Democrat'],
                                 trump_vote=votes['Donald Trump, the Republican'],
                                 n=n))
    
print('## CALIFORNIA CA-WEIGHTED ##')
weighted_n = int(np.round(survey_['weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_trump_biden'].value_counts(normalize=True) * survey_.groupby('vote_trump_biden')['weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result_tb(**calc_result_tb(biden_vote=votes['Joe Biden, the Democrat'],
                                 trump_vote=votes['Donald Trump, the Republican'],
                                 n=weighted_n))

print('## CALIFORNIA CA-WEIGHTED + RV ##')
rv_weighted_n = int(np.round(survey_['rv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_trump_biden'].value_counts(normalize=True) * survey_.groupby('vote_trump_biden')['rv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result_tb(**calc_result_tb(biden_vote=votes['Joe Biden, the Democrat'],
                                 trump_vote=votes['Donald Trump, the Republican'],
                                 n=rv_weighted_n))

print('## CALIFORNIA CA-WEIGHTED + LV ##')
lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
votes = survey_['vote_trump_biden'].value_counts(normalize=True) * survey_.groupby('vote_trump_biden')['lv_weight'].mean() * 100
votes = votes[options] * (100 / votes[options].sum())
print(votes)
print_result_tb(**calc_result_tb(biden_vote=votes['Joe Biden, the Democrat'],
                                 trump_vote=votes['Donald Trump, the Republican'],
                                 n=lv_weighted_n))

## NATIONAL CA-WEIGHTED ##
Joe Biden, the Democrat         64.508308
Donald Trump, the Republican    26.319267
Another candidate                3.964050
Not decided                      5.208376
dtype: float64
Result Biden 38.2 (80% CI: 31.6 to 44.9) (Weighted N=1781) (raw_moe=2.8pts, margin=14.5pts, sigma=5.6pts) (Biden >99.9% likely to win)
-
## NATIONAL CA-WEIGHTED + RV ##
Joe Biden, the Democrat         64.596909
Donald Trump, the Republican    27.196818
Another candidate                4.133003
Not decided                      4.073270
dtype: float64
Result Biden 37.4 (80% CI: 30.9 to 43.9) (Weighted N=1747) (raw_moe=2.8pts, margin=14.1pts, sigma=5.5pts) (Biden >99.9% likely to win)
-
## NATIONAL CA-WEIGHTED + LV ##
Joe Biden, the Democrat         65.345466
Donald Trump, the Republican    26.958914
Another candidate                4.043662
Not decided                      3.651958
dtype: float64
Result Biden 38.4 (80% CI: 31.9 to 44.8) (Weighted N=1758) (raw_moe=2.8pts, margin=13.

## California Propositions

In [37]:
options = ['Vote for / Support', 'Vote against / Oppose', 'Don’t know / Undecided']
ca_measures = [c for c in ca_national_survey.columns if 'vote_measure_ca_' in c and 'meta' not in c]

for measure in ca_measures:        
    survey_ = ca_national_survey.loc[ca_national_survey[measure].isin(options)].copy()
    survey_['weight'] = normalize_weights(survey_['weight'])
    survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
    survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
    
    print('## {} NATIONAL CA-WEIGHTED ##'.format(measure))
    weighted_n = int(np.round(survey_['weight'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=weighted_n))

    print('## {} NATIONAL CA-WEIGHTED + LV ##'.format(measure))
    lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['lv_weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=lv_weighted_n))
    
    if measure != 'vote_measure_100pct_income_tax':
        survey_ = ca_national_survey.loc[ca_national_survey[measure].isin(options)].copy()
        survey_.loc[survey_['vote_measure_100pct_income_tax'] == 'Vote for / Support', measure] = 'Don’t know / Undecided'
        survey_['weight'] = normalize_weights(survey_['weight'])
        survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
        survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])

        print('## {} NATIONAL CORRECTED + CA-WEIGHTED + LV ##'.format(measure))
        lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
        votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['lv_weight'].mean() * 100
        votes = votes[options] * (100 / votes[options].sum())
        print(votes)
        print_result(**calc_result(for_vote=votes['Vote for / Support'],
                                   against_vote=votes['Vote against / Oppose'],
                                   n=lv_weighted_n))

    survey_ = ca_state_survey.loc[ca_state_survey[measure].isin(options)].copy()
    survey_['weight'] = normalize_weights(survey_['weight'])
    survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
    survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
    
    print('## {} CALI UNWEIGHTED ##'.format(measure))
    n = len(survey_)
    votes = survey_[measure].value_counts(normalize=True) * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=n))
    
    print('## {} CALI CA-WEIGHTED ##'.format(measure))
    weighted_n = int(np.round(survey_['weight'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=weighted_n))

    print('## {} CALI CA-WEIGHTED + LV ##'.format(measure))
    lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['lv_weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=lv_weighted_n))
    
    if measure != 'vote_measure_100pct_income_tax':
        survey_ = ca_state_survey.loc[ca_state_survey[measure].isin(options)].copy()
        survey_ = survey_.loc[survey_['state'] == 'California']
        survey_.loc[survey_['vote_measure_100pct_income_tax'] == 'Vote for / Support', measure] = 'Don’t know / Undecided'
        survey_['weight'] = normalize_weights(survey_['weight'])
        survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
        survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
        
        print('## {} CALI CORRECTED + CA-WEIGHTED + LV ##'.format(measure))
        lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
        votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['lv_weight'].mean() * 100
        votes = votes[options] * (100 / votes[options].sum())
        print(votes)
        print_result(**calc_result(for_vote=votes['Vote for / Support'],
                                   against_vote=votes['Vote against / Oppose'],
                                   n=lv_weighted_n))

    print('-')

## vote_measure_ca_prop16_affirmative_action NATIONAL CA-WEIGHTED ##
Vote for / Support        41.460895
Vote against / Oppose     43.656905
Don’t know / Undecided    14.882199
dtype: float64
Result 45.2 (80% CI: 30.2 to 60.2) (N=666) (raw_moe=4.9pts, margin=22.9pts, sigma=17.8pts) (39.3% likely to pass)
45.2 (80% CI: 30.2 to 60.2) (39.3%)
-
## vote_measure_ca_prop16_affirmative_action NATIONAL CA-WEIGHTED + LV ##
Vote for / Support        42.520048
Vote against / Oppose     44.962144
Don’t know / Undecided    12.517808
dtype: float64
Result 45.6 (80% CI: 31.2 to 60.2) (N=628) (raw_moe=5.1pts, margin=22.1pts, sigma=17.2pts) (40.0% likely to pass)
45.6 (80% CI: 31.2 to 60.2) (40.0%)
-
## vote_measure_ca_prop16_affirmative_action NATIONAL CORRECTED + CA-WEIGHTED + LV ##
Vote for / Support        31.802845
Vote against / Oppose     34.745350
Don’t know / Undecided    33.451805
dtype: float64
Result 40.2 (80% CI: 20.4 to 60.0) (N=628) (raw_moe=4.8pts, margin=30.2pts, sigma=23.6pts) (33.8% 

Result 59.3 (80% CI: 41.9 to 76.8) (N=641) (raw_moe=4.6pts, margin=26.6pts, sigma=20.7pts) (67.4% likely to pass)
59.3 (80% CI: 41.9 to 76.8) (67.4%)
-
## vote_measure_ca_prop15_property_tax CALI UNWEIGHTED ##
Vote for / Support        59.246575
Vote against / Oppose     23.972603
Don’t know / Undecided    16.780822
Name: vote_measure_ca_prop15_property_tax, dtype: float64
Result 63.4 (80% CI: 48.6 to 78.3) (N=876) (raw_moe=4.0pts, margin=22.7pts, sigma=17.7pts) (77.6% likely to pass)
63.4 (80% CI: 48.6 to 78.3) (77.6%)
-
## vote_measure_ca_prop15_property_tax CALI CA-WEIGHTED ##
Vote for / Support        57.044071
Vote against / Oppose     27.878193
Don’t know / Undecided    15.077736
dtype: float64
Result 60.8 (80% CI: 44.7 to 76.9) (N=348) (raw_moe=6.5pts, margin=24.5pts, sigma=19.1pts) (71.4% likely to pass)
60.8 (80% CI: 44.7 to 76.9) (71.4%)
-
## vote_measure_ca_prop15_property_tax CALI CA-WEIGHTED + LV ##
Vote for / Support        60.629319
Vote against / Oppose     27.520574
Do

Result 61.5 (80% CI: 46.8 to 76.3) (N=330) (raw_moe=6.7pts, margin=22.5pts, sigma=17.5pts) (74.5% likely to pass)
61.5 (80% CI: 46.8 to 76.3) (74.5%)
-
## vote_measure_ca_prop25_bail CALI CORRECTED + CA-WEIGHTED + LV ##
Vote for / Support        50.273168
Vote against / Oppose     26.431203
Don’t know / Undecided    23.295629
dtype: float64
Result 56.1 (80% CI: 37.7 to 74.5) (N=330) (raw_moe=6.6pts, margin=27.9pts, sigma=21.8pts) (61.0% likely to pass)
56.1 (80% CI: 37.7 to 74.5) (61.0%)
-
-
## vote_measure_ca_prop17_felon_vote NATIONAL CA-WEIGHTED ##
Vote for / Support        65.080835
Vote against / Oppose     23.865957
Don’t know / Undecided    11.053208
dtype: float64
Result 67.8 (80% CI: 54.1 to 81.6) (N=636) (raw_moe=4.6pts, margin=21.0pts, sigma=16.4pts) (86.2% likely to pass)
67.8 (80% CI: 54.1 to 81.6) (86.2%)
-
## vote_measure_ca_prop17_felon_vote NATIONAL CA-WEIGHTED + LV ##
Vote for / Support        67.666994
Vote against / Oppose     25.525674
Don’t know / Undecided     6.